In [ ]:
# %%capture 
# !pip install -U tokenizers
# !pip install -U transformers
# !pip install -U tokenizers
# !pip install -U transformers

In [1]:
import datasets
import sys

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

sys.path.append('../../../')
from transformers.models.bert.tokenization_bert import BertTokenizer
from transformers.models.bert.modeling_linbert import BertForSequenceClassification

import torch

In [2]:
torch.cuda.device_count()

1

In [3]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding


In [4]:

# from transformers_modified.models.bert.tokenization_bert import BertTokenizer
# BertModelForSequenceClassification
import torch
import numpy as np

In [5]:
model = BertForSequenceClassification.from_pretrained('M-FAC/bert-tiny-finetuned-sst2', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('M-FAC/bert-tiny-finetuned-sst2')

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [6]:
model.linearize(32, 41)

linearized


In [7]:
def create_data(tokenizer):
    #train_set = datasets.load_dataset('sst2', split='train').remove_columns(['idx'])
    val_set = datasets.load_dataset('sst2', split='validation').remove_columns(['idx'])

    dynamic_padding = True

    def tokenize_func(examples):
        return tokenizer(examples["sentence"], max_length=32, padding='max_length', truncation=True)

  #  encoded_dataset_train = train_set.map(tokenize_func, batched=True)
    encoded_dataset_test = val_set.map(tokenize_func, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer)

    return encoded_dataset_test, data_collator

In [8]:
encoded_dataset_train, data_collator = create_data(tokenizer)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [9]:
#model.linearize(seq_len=32, k=41)

In [10]:
out = model.forward(input_ids=torch.tensor(encoded_dataset_train[0]['input_ids']).unsqueeze(0))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


before
torch.Size([1, 2, 32, 64]) torch.Size([32, 41])
torch.Size([1, 2, 32, 64]) torch.Size([32, 41])
after
torch.Size([1, 2, 41, 64])
torch.Size([1, 2, 41, 64])
att scores
torch.Size([1, 2, 32, 41])
no mask
before
torch.Size([1, 2, 32, 64]) torch.Size([32, 41])
torch.Size([1, 2, 32, 64]) torch.Size([32, 41])
after
torch.Size([1, 2, 41, 64])
torch.Size([1, 2, 41, 64])
att scores
torch.Size([1, 2, 32, 41])
no mask


In [11]:
out

SequenceClassifierOutput(loss=None, logits=tensor([[-0.2158,  0.0623]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
key = torch.rand(1,2,32,64)
E = torch.rand(32, 41)

In [12]:
(key.transpose(-1, -2) @ E).size()

torch.Size([1, 2, 64, 41])

In [14]:
E @ key.transpose(-1, -2)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x64 and 32x41)

In [12]:
for layer in model.bert.encoder.layer:
    print(layer.attention.self.linearize)

AttributeError: 'BertSelfAttention' object has no attribute 'linearize'

In [10]:
metric = datasets.load_metric('accuracy')

def compute_metrics(eval_pred):
	predictions, labels = eval_pred
	predictions = np.argmax(predictions, axis=1)
	return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_228402/2290328016.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('accuracy')


In [11]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="epoch",
  #  eval_steps=500,
    seed=42,
    save_strategy = "epoch",
    save_total_limit=5,
    logging_strategy="epoch",
    report_to="all",
   # logging_steps=500
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [12]:
for param in model.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.230800,0.212387,0.908257
2,0.110700,0.251804,0.923165
3,0.073200,0.254554,0.923165
4,0.047700,0.279523,0.925459
5,0.030800,0.314340,0.925459


TrainOutput(global_step=2635, training_loss=0.09864934237225007, metrics={'train_runtime': 791.5703, 'train_samples_per_second': 425.414, 'train_steps_per_second': 3.329, 'total_flos': 8933858801431800.0, 'train_loss': 0.09864934237225007, 'epoch': 5.0})

In [14]:
test_set = datasets.load_dataset('sst2', split='test').remove_columns(['idx'])

dynamic_padding = True

def tokenize_func(examples):
    return tokenizer(examples["sentence"], truncation=True)  # max_length=512,  padding=True

encoded_dataset_test2 = test_set.map(tokenize_func, batched=True)

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [25]:
trainer.state.best_metric, trainer.state.best_model_checkpoint

(0.9254587155963303, './results/checkpoint-2108')